In [1]:
import numpy as np

def external_potential(x,y,z):
    return (x**2 + y**2 + z**2) /2

def init_psi(x_in, y_in, z_in):
    return np.random.uniform(0, 1, np.shape(x_in)) + 0.j

def laplacian(psi):
    psi_k = np.fft.fftn(psi)
    lap = np.fft.ifftn(psi_k * sumk2)
    return lap
    
def kinetic_energy(psi):
    global d3r, sumk2
    l = laplacian(psi) * np.conj(psi)
    e_kin = (np.sum(l) * d3r).real / 2
    return e_kin

def T2_operator(psi):
    global pot_ext, d3r
    kinprop = np.exp(-dt * sumk2/2) 
    psi_t = np.exp(-0.5 * pot_ext * dt) * psi    
    psi_k = np.fft.fftn(psi_t)
    psi_t = np.fft.ifftn(kinprop * psi_k)
    psi_t = np.exp(-0.5 * pot_ext * dt) * psi_t
    return psi_t

def pot_energy(psi):
    global d3r, pot_ext
    den_t = np.absolute(psi)**2
    pot_en=d3r*np.sum(pot_ext*den_t)
    return pot_en

def overlap(psi_1, psi_2):
    return d3r * np.sum(np.conj(psi_1) * psi_2  )

def create_matrix(psi_all):
    size = len(psi_all)
    matrix = np.ones( (size, size), dtype=np.complex128 )
    for i in range(size):
        for j in range(size):
            matrix[i,j] = overlap(psi_all[i], psi_all[j])
    return matrix

def propagate(psis): ##uzme svaki psi i propagira ga
    psis_out = np.copy(psis)
    for i, psi in enumerate(psis):
        psis_out[i]=T2_operator(psi)
    return psis_out

def ortho(psis):
    m=create_matrix(psis)
    eigen_vals, eigen_vecs = np.linalg.eig(m)
    psi_all_new = np.copy(psis)
    n_size=len(psis)
    for j in range(n_size):
        tmp = 0
        for i in range(n_size):
            tmp += eigen_vecs[i][j] * psi_all[i] #* psi_all_rotated[i]
        psi_all_new[j] = tmp/ np.sqrt(eigen_vals[j])
    for i in range(n_size):
        den_t = np.absolute(psi_all_new[i])**2
        normtemp = np.sum(den_t) * d3r
        psi_all_new[i] *= np.sqrt(1. / normtemp)
    return psi_all_new

arr=32
nxyz = np.array([arr, arr, arr])
L = np.array([20, 20, 20])
tot_time = 10

LHalf = L / 2
x = np.linspace( -LHalf[0], LHalf[0],  nxyz[0],  endpoint=False )
y = np.linspace( -LHalf[1], LHalf[1],  nxyz[1],  endpoint=False )
z = np.linspace( -LHalf[2], LHalf[2],  nxyz[2],  endpoint=False )
dx = L / nxyz
d3r = np.prod(dx)
x, y, z = np.meshgrid(x, y, z, indexing='ij')

kx = np.fft.fftfreq(nxyz[0], dx[0] / ( 2 * np.pi))
ky = np.fft.fftfreq(nxyz[1], dx[1] / ( 2 * np.pi))
kz = np.fft.fftfreq(nxyz[2], dx[2] / ( 2 * np.pi))
kx, ky, kz = np.meshgrid(ky, ky, kz, indexing='ij')

sumk2 = kx**2 + ky**2 + kz**2
pot_ext = external_potential(x, y, z)

In [2]:
dt=0.01
n_iter=1000
n_states = 10

###

psi_all = []
for i in range(n_states):
    psi_all.append(init_psi(x,y,z))
pot_ear=[]
kin_ear=[]
for epoch in range(n_iter):
    psi_all = propagate(psi_all)
    psi_all = ortho(psi_all)

e_round = np.zeros(n_states)
ek_round = np.zeros(n_states)
n_indx = np.arange(0, 10, 1)
en_analytic = 0.5 * (n_indx + 3./2)

for i in range(n_states):
  pot_ear.append(pot_energy(psi_all[i]))
  e_round[i] = round(pot_ear[i],2)
  kin_ear.append(kinetic_energy(psi_all[i]))
  ek_round[i] = round(kin_ear[i],2)

print("E_numerical = ", pot_ear)
print("E_analytic = ", en_analytic)
indx_sorted = np.argsort(e_round)
e_roundsort = e_round[indx_sorted]
print("E_numerical; rounded, sorted = ", e_roundsort)
print("\n")
print("Ek_numerical =", kin_ear)
ek_roundsort = ek_round[indx_sorted]
print("Ek_numerical; rounded, sorted = ", ek_roundsort)
unique, counts = np.unique(ek_round, return_counts=True)
dict(zip(unique, counts))

E_numerical =  [0.7499906228543447, 1.2499844099235171, 1.2499844119856172, 1.2499844113581136, 1.749977777191913, 1.7499779622792448, 1.7499774019165775, 1.7499777482379322, 1.7499776806634904, 1.7499780192347365]
E_analytic =  [0.75 1.25 1.75 2.25 2.75 3.25 3.75 4.25 4.75 5.25]
E_numerical; rounded, sorted =  [0.75 1.25 1.25 1.25 1.75 1.75 1.75 1.75 1.75 1.75]


Ek_numerical = [0.7500093770829643, 1.2500155929961458, 1.2500155909339952, 1.250015591561515, 1.7500222061316144, 1.7500220346649868, 1.7500225477868503, 1.7500222380287649, 1.750022291655731, 1.7500219903303602]
Ek_numerical; rounded, sorted =  [0.75 1.25 1.25 1.25 1.75 1.75 1.75 1.75 1.75 1.75]


{0.75: 1, 1.25: 3, 1.75: 6}